In [7]:
import pandas as pd
import numpy as np

def createTaskQueues2(taskInfo):
    
    taskQueues = {}
    for time in range(90000,113100,300):
        taskQueues[time] = []
    time = 90000
    for taskIdx in range(len(taskInfo['Time'])):
        if(taskInfo['Time'][taskIdx] == time):
            taskQueues[time].append(taskIdx)
            if(taskInfo['ExecutionTime'][taskIdx] > 300):
                taskQueues[time+300].append(taskIdx)
        else:
            time = time + 300
    return taskQueues

def powerConsumption(myVmSpace):
    c = 5
    a = 100
    b = 200
    threshold = 0.5
    
    Pwr_dy = 0
    Pwr_st = 0
    Pwr = 0
    
    for core in myVmSpace[0]:
        coreUsage = 1-core/7
        if(coreUsage > 0):
            Pwr_st = c
        if(coreUsage < threshold):
            Pwr_dy = a*coreUsage
        else:
            Pwr_dy = a*threshold + b*(coreUsage-threshold)*(coreUsage-threshold)
        Pwr = Pwr + Pwr_dy + Pwr_st
    return Pwr

def roundRobin(taskQueue,taskInfo,myVmSpace):
               
    VM_iter = 0
    #will iterate through all possible starting VMs, incriments by one each time
    VM_jter = 0
    #incrimenter used when task does not fit in the first potentially available VM
    
    rejectQueue = []
               
    for i in taskQueue:
        if(taskInfo['NrmlTaskCores'][i] <= myVmSpace[0][VM_iter] and taskInfo['NrmlTaskMem'][i] <= myVmSpace[1][VM_iter]):
            #if the number of cores and memory space the task requires is less than the available for the VMS
            #then we can allocate the task to the current VM
            myVmSpace[0][VM_iter]  = myVmSpace[0][VM_iter] - taskInfo['NrmlTaskCores'][i]
            myVmSpace[1][VM_iter]  = myVmSpace[1][VM_iter] - taskInfo['NrmlTaskMem'][i]
            #now that the task is allocated, we have to update the amount of free CPU and memory space
        else:
            allocated = False
            VM_jter = VM_iter+1
            #move to the next VM
            if(VM_jter == 100):
                #make sure to loop back
                VM_jter = 0
            while(not allocated and VM_jter!=VM_iter):
                #if it is allocated then don't keep looking
                #if you've looked at all possible VMs and its not allocated, then it never will be so end loop
                if(taskInfo['NrmlTaskCores'][i] <= myVmSpace[0][VM_jter] and taskInfo['NrmlTaskMem'][i] <= myVmSpace[1][VM_jter]):
                    #if the number of cores and memory space the task requires is less than the available for the VMS
                    #then we can allocate the task to the current VM
                    myVmSpace[0][VM_jter] = myVmSpace[0][VM_jter] - taskInfo['NrmlTaskCores'][i]
                    myVmSpace[1][VM_jter] = myVmSpace[1][VM_jter] - taskInfo['NrmlTaskMem'][i]
                    #now that the task is allocated, we have to update the amount of free CPU and memory space
                    allocated = True
                    #allocate the task 
                VM_jter = VM_jter+1
                #move to the next VM if its not allocated to the last one
                if(VM_jter == 100):
                    VM_jter = 0
            if(not allocated):
                rejectQueue.append(taskInfo['TaskID'][i])
                #if the while loop finishes and the task is still not allocated
                #then it cycled through all VMs and did not fit any, so it cannot be allocated and should be added to reject queue
        VM_iter = VM_iter+1
        if(VM_iter == 100):
            VM_iter = 0

    return rejectQueue
    
def main():
    taskData = pd.read_csv('google-cluster-data-1.csv',sep=' ') 
    #gathering csv data into a python dictionary
    taskInfo = {'TaskID':taskData['TaskID'],'NrmlTaskCores':taskData['NrmlTaskCores'],'NrmlTaskMem':taskData['NrmlTaskMem'],'Time':taskData['Time'],'ExecutionTime':300*(taskData['TaskID']%2+1)}
    #created a new python dictionary with more intuitive formatting
    
    np.save('taskID.npy',taskInfo['TaskID'])
    np.save('taskCPU.npy',taskInfo['NrmlTaskCores'])
    np.save('taskMEM.npy',taskInfo['NrmlTaskMem'])
    np.save('exe_time_1.npy',taskInfo['ExecutionTime'])
    
    timeQ = 300

    rejectQueue = [] 
    #queue of tasks that could not fit into any VM for all times
    rejectQueue_t = []
    
    taskQueues = createTaskQueues2(taskInfo)
    
    Pwr_dy_t = {}
    Pwr_st_t = {}
    Pwr_t = {}
    Cost_t = {}
    
    for time in range(90000,113100,300):
        Pwr_dy_t[time] = []
        Pwr_st_t[time] = []
        Pwr_t[time] = []
        Cost_t[time] = []
    
    Price_t = [0.5, 0.5, 0.6, 0.6, 0.6, 0.7, 0.7, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8]
    totalCost = 0
    totalPwr = 0
    
    for time in range(90000,113100,timeQ):
        taskQueue_t = taskQueues[time]
        myVmSpace = [100*[7],100*[11]]
        #reinitialize VMs for new time
        rejectQueue_t = roundRobin(taskQueue_t,taskInfo,myVmSpace)
        if(rejectQueue_t != []):
            rejectQueue.append(rejectQueue_t)
        Pwr_t[time] = powerConsumption(myVmSpace)
        totalPwr = totalPwr + Pwr_t[time]
        prcIdx = int((time - 90000)/3600)
        Cost_t[time] = Price_t[prcIdx]*Pwr_t[time]
        totalCost = totalCost + Cost_t[time]
            
    np.save('VMs_1.npy',myVmSpace)
                
    print("Number of rejected tasks is: {}".format(len(rejectQueue)))
    #print out the number of tasks that were rejected by the VMs
    print("Total power use of tasks is: {}".format(totalPwr))
    print("Total cost of tasks is: {}".format(totalCost))
    
    if(rejectQueue != []):
        np.save('taskReject_1.npy',rejectQueue)
        
if __name__=="__main__":
    main()

Number of rejected tasks is: 0
Total power use of tasks is: 585318.7535873706
Total cost of tasks is: 345528.0224250627


In [11]:
import pandas as pd
import numpy as np

def createTaskQueues10(taskInfo):
    
    taskQueues = {}
    for time in range(90000,115500,300):
        taskQueues[time] = []
    time = 90000
    for taskIdx in range(len(taskInfo['Time'])):
        if(taskInfo['Time'][taskIdx] == time):
            exTime = taskInfo['ExecutionTime'][taskIdx]
            timeCnt = 0
            while(exTime > 0):
                taskQueues[time+timeCnt].append(taskIdx)
                timeCnt = timeCnt + 300
                exTime = exTime - 300
        else:
            time = time + 300
    return taskQueues
            
def powerConsumption(myVmSpace):
    c = 5
    a = 100
    b = 200
    threshold = 0.5
    
    Pwr_dy = 0
    Pwr_st = 0
    Pwr = 0
    
    for core in myVmSpace[0]:
        coreUsage = 1-core/25
        if(coreUsage > 0):
            Pwr_st = c
        if(coreUsage < threshold):
            Pwr_dy = a*coreUsage
        else:
            Pwr_dy = a*threshold + b*(coreUsage-threshold)*(coreUsage-threshold)
        Pwr = Pwr + Pwr_dy + Pwr_st
    return Pwr

def roundRobin(taskQueue,taskInfo,myVmSpace):
               
    VM_iter = 0
    #will iterate through all possible starting VMs, incriments by one each time
    VM_jter = 0
    #incrimenter used when task does not fit in the first potentially available VM
    
    rejectQueue = []
               
    for i in taskQueue:
        if(taskInfo['NrmlTaskCores'][i] <= myVmSpace[0][VM_iter] and taskInfo['NrmlTaskMem'][i] <= myVmSpace[1][VM_iter]):
            #if the number of cores and memory space the task requires is less than the available for the VMS
            #then we can allocate the task to the current VM
            myVmSpace[0][VM_iter]  = myVmSpace[0][VM_iter] - taskInfo['NrmlTaskCores'][i]
            myVmSpace[1][VM_iter]  = myVmSpace[1][VM_iter] - taskInfo['NrmlTaskMem'][i]
            #now that the task is allocated, we have to update the amount of free CPU and memory space
        else:
            allocated = False
            VM_jter = VM_iter+1
            #move to the next VM
            if(VM_jter == 100):
                #make sure to loop back
                VM_jter = 0
            while(not allocated and VM_jter!=VM_iter):
                #if it is allocated then don't keep looking
                #if you've looked at all possible VMs and its not allocated, then it never will be so end loop
                if(taskInfo['NrmlTaskCores'][i] <= myVmSpace[0][VM_jter] and taskInfo['NrmlTaskMem'][i] <= myVmSpace[1][VM_jter]):
                    #if the number of cores and memory space the task requires is less than the available for the VMS
                    #then we can allocate the task to the current VM
                    myVmSpace[0][VM_jter] = myVmSpace[0][VM_jter] - taskInfo['NrmlTaskCores'][i]
                    myVmSpace[1][VM_jter] = myVmSpace[1][VM_jter] - taskInfo['NrmlTaskMem'][i]
                    #now that the task is allocated, we have to update the amount of free CPU and memory space
                    allocated = True
                    #allocate the task 
                VM_jter = VM_jter+1
                #move to the next VM if its not allocated to the last one
                if(VM_jter == 100):
                    VM_jter = 0
            if(not allocated):
                rejectQueue.append(taskInfo['TaskID'][i])
                #if the while loop finishes and the task is still not allocated
                #then it cycled through all VMs and did not fit any, so it cannot be allocated and should be added to reject queue
        VM_iter = VM_iter+1
        if(VM_iter == 100):
            VM_iter = 0

    return rejectQueue
    
def main():
    taskData = pd.read_csv('google-cluster-data-1.csv',sep=' ') 
    #gathering csv data into a python dictionary
    taskInfo = {'TaskID':taskData['TaskID'],'NrmlTaskCores':taskData['NrmlTaskCores'],'NrmlTaskMem':taskData['NrmlTaskMem'],'Time':taskData['Time'],'ExecutionTime':300*(taskData['TaskID']%10+1)}
    #created a new python dictionary with more intuitive formatting
    
    np.save('taskID.npy',taskInfo['TaskID'])
    np.save('taskCPU.npy',taskInfo['NrmlTaskCores'])
    np.save('taskMEM.npy',taskInfo['NrmlTaskMem'])
    np.save('exe_time_2.npy',taskInfo['ExecutionTime'])
    
    timeQ = 300

    rejectQueue = [] 
    rejectQueue_t = []
    #queue of tasks that could not fit into any VM for all times
    
    taskQueues = createTaskQueues10(taskInfo)
    
    Pwr_dy_t = {}
    Pwr_st_t = {}
    Pwr_t = {}
    Cost_t = {}
    
    for time in range(90000,113100,300):
        Pwr_dy_t[time] = []
        Pwr_st_t[time] = []
        Pwr_t[time] = []
        Cost_t[time] = []
    
    Price_t = [0.5, 0.5, 0.6, 0.6, 0.6, 0.7, 0.7, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8]
    totalCost = 0
    totalPwr = 0
    
    for time in range(90000,115500,timeQ):
        taskQueue_t = taskQueues[time]
        myVmSpace = [100*[25],100*[40]]
        #reinitialize VMs for new time
        rejectQueue_t = roundRobin(taskQueue_t,taskInfo,myVmSpace)
        if(rejectQueue_t != []):
            rejectQueue.append(rejectQueue_t)
        Pwr_t[time] = powerConsumption(myVmSpace)
        totalPwr = totalPwr + Pwr_t[time]
        prcIdx = int((time - 90000)/3600)
        Cost_t[time] = Price_t[prcIdx]*Pwr_t[time]
        totalCost = totalCost + Cost_t[time]
            
    np.save('VMs_2.npy',myVmSpace)
                
    print("Number of rejected tasks is: {}".format(len(rejectQueue)))
    #print out the number of tasks that were rejected by the VMs
    print("Total power use of tasks is: {}".format(totalPwr))
    print("Total cost of tasks is: {}".format(totalCost))
    
    if(rejectQueue != []):
        np.save('taskReject_2.npy',rejectQueue)
    
if __name__=="__main__":
    main()

Number of rejected tasks is: 0
Total power use of tasks is: 617857.1432187563
Total cost of tasks is: 369147.2679653163


# Program Description

The both programs have the same structure and can be divided into three steps: generating the task queue for each time quantum, running the round robin algorithm for each time quantum, and calculating the cost of the scheduled tasks. 

The task queues are generated for all time quantums before the round robin algorithm allocates the tasks to VMs. For each time quantum, the task queue-generating algorithmchecks if the task is ready at that time. If it is, the task is added to the queue for that time. If the task lasts multiple time quantums, then it is added to the queue of the following time quantums until it's spread over enough time quantums that it will be able to complete. However, if a task is not ready at the current time, the time is incrimented and the steps are repeated to fill the next task queue. 

Then, for each time quantum, the round robin alogorithm runs on the task queue for that time. The round robin algorithm will then allocated as many tasks as possible to the available VMs. For each task, the algorithm checks if the next available VM can fit those tasks. If so, then the task is allocated and the CPU and MEM space it uses is removed from the available space. Otherwise, the algorithm goes through all other VMs to see if it can fit in their spaces. If so, it is allocated to that VM. If the task cannot fit in any VM, it is appended to the reject queue.

At each time quantum, the powerConsumption algorithm then calculates the power used by the VMs. The power for each VM is calculated as the sum of the dynamic and static powers for that VM. The power for each VM is then summed to get the total power consumed at that time. This total is then returned to the main program. 

The main program will multiply the power at each time by a constant dependent on that time to get the cost at that time. The power and cost can then be summed across all times. This produces the total power and cost used by the tasks when allocated using RR. 